# Urban Climate Homework 1

In [2]:
from data_man import data_explore
import pandas as pd
import plotly
import plotly.graph_objects as go

In [3]:

df_urban = pd.read_csv("save_BWI2.csv")
df_rural = pd.read_csv("save_DMH2.csv")

rural_df = data_explore(df_rural, "rural").data_celaning()
urban_df = data_explore(df_urban, "urban").data_celaning()

print(rural_df)
print(urban_df)



year         False
month        False
day          False
Mean T        True
Maximum T     True
Minimum T     True
dtype: bool
Column name =  year
Column name =  month
Column name =  day
Column name =  Mean T
Column name =  Maximum T
Column name =  Minimum T
year         False
month        False
day          False
Mean T       False
Maximum T    False
Minimum T    False
dtype: bool
year         False
month        False
day          False
Mean T       False
Maximum T    False
Minimum T    False
dtype: bool
Column name =  year
Column name =  month
Column name =  day
Column name =  Mean T
Column name =  Maximum T
Column name =  Minimum T
year         False
month        False
day          False
Mean T       False
Maximum T    False
Minimum T    False
dtype: bool
      year  month  day  Mean T  Maximum T  Minimum T
0     2000      1    2    54.0         68         40
1     2000      1    3    60.0         66         54
2     2000      1    4    59.0         70         48
3     2000      1   

1.1, Calculate the urban heat island intensity using daily mean, maximum, and minimum temperatures for each day and then average them to each month (no figure or value is needed). Then average the monthly results from 2000 to 2016. For example, you should average all the January results from 2000 to 2016 to obtain an averaged January urban heat island effect. Plot the averaged urban heat islands as a function of month (you can think of this as the averaged seasonal cycle of urban heat island). Use different colors or markers for daily mean, maximum, and minimum temperatures but plot them on the same figure.


In order to calculate Urban Heat Island Intensity we can use: 

$$ \Delta T = T_{urban} - T_{rural}$$

In [15]:

"""
First we can get the Urban Heat Island per day

"""

years = 2000

for ii in range(17):

    print(years)
    df_rural_year = df_rural.loc[df_rural['year'] == years]
    years = years + 1

    for jj in range(12):

        jj = jj + 1
        df_rural_month = df_rural_year.loc[df_rural_year['month'] == jj]
        print(df_rural_month)
        print(df_rural_month.loc[:, 'Mean T'].mean())
        




2000
    year  month  day  Mean T  Maximum T  Minimum T
0   2000      1    2    54.0         68         40
1   2000      1    3    60.0         66         54
2   2000      1    4    59.0         70         48
3   2000      1    5    42.0         48         36
4   2000      1    6    40.0         47         32
5   2000      1    7    43.0         52         34
6   2000      1    8    39.0         46         31
7   2000      1    9    44.0         48         40
8   2000      1   10    48.0         53         43
9   2000      1   11    48.0         55         41
10  2000      1   12    43.0         49         37
11  2000      1   13    44.0         58         30
12  2000      1   14    27.0         31         23
13  2000      1   15    29.0         36         22
14  2000      1   16    46.0         60         31
15  2000      1   17    24.0         30         18
16  2000      1   18    19.0         24         14
17  2000      1   19    30.0         38         21
18  2000      1   20    30